# Notebook for manuscript
# Trizzino, Park et al (2016)

## Following scripts or their modified versions are used for RNA-seq data processing pipelines
## 2015 - 2016

RNA-seq data processing
Original scripts by MT, modified and additionally commented by YP

In [ ]:
%%writefile 'RNASEQ.PROCESSING.sh'

#!/bin/bash

# Marco Trizzino

##########RNA-seq data processing

##FOR EACH INDIVIDUAL SAMPLE WE HAD SEQUENCES FROM TWO LANES:

#CALLING THE VARIABLES
echo $1 # Read RNA-seq sample lane 1
echo $2 # Read RNA-seq sample lane 2
echo $3 # reference genome
echo $4 # Sample output name 
echo $5 # rsem reference 

#PERFORMING QC OF THE SEQUENCES WITH FASTQC
fastqc $1.fastq -o /OUTPUT_PATH
fastqc $2.fastq -o /OUTPUT_PATH

#TRIMMING ADAPTERS FROM RNA-seq reads
trim_galore --cutadapt /cutadapt -stringency 5 -length 50 -o /OUTPUT_PATH/ -q 20 $1.fastq $2.fastq

#STAR 2-PASS ALIGNMENT

STAR --limitGenomeGenerateRAM 240000000000 \\
--genomeDir $3 \\
--readFilesIn $1_trimmed.fq \\
--outSAMtype SAM \\
--quantMode TranscriptomeSAM \\
--outFilterMultimapNmax 10 \\
--outFilterMismatchNmax 10 \\
--outFilterMismatchNoverLmax 0.3 \\
--alignIntronMin 21 \\
--alignIntronMax 0 \\
--alignMatesGapMax 0 \\
--alignSJoverhangMin 5 \\
--runThreadN 12 \\
--twopassMode Basic \\
--twopass1readsN 60000000 \\
--sjdbOverhang 100 \\
--outFileNamePrefix $4_lane1.

STAR --limitGenomeGenerateRAM 240000000000 \\
--genomeDir $3 \\
--readFilesIn $2_trimmed.fq \\
--outSAMtype SAM \\
--quantMode TranscriptomeSAM \\
--outFilterMultimapNmax 10 \\
--outFilterMismatchNmax 10 \\
--outFilterMismatchNoverLmax 0.3 \\
--alignIntronMin 21 \\
--alignIntronMax 0 \\
--alignMatesGapMax 0 \\
--alignSJoverhangMin 5 \\
--runThreadN 12 \\
--twopassMode Basic \\
--twopass1readsN 60000000 \\
--sjdbOverhang 100 \\
--outFileNamePrefix $4_lane2.

mv $4_lane1.Aligned.out.sam $4_lane1_STAR.sam
mv $4_lane2.Aligned.out.sam $4_lane2_STAR.sam

#CONVERTING SAM IN BAM
samtools view -@ 8 -Sb $4_lane1_STAR.sam > $4_lane1_STAR.bam
samtools view -@ 8 -Sb $4_lane2_STAR.sam > $4_lane2_STAR.bam

#THE FOLLOWING COMMANDS SORT THE READS BASED ON CHROMOSOMES COORDINATES
samtools sort -@ 8 -T $4_lane1_STAR.bam -O bam -o $4_lane1_STAR.bam $4_lane1_STAR_srt.bam 
samtools sort -@ 8 -T $4_lane2_STAR.bam -O bam -o $4_lane2_STAR.bam $4_lane2_STAR_srt.bam 

# merge BAM
samtools merge -f -@ 8 $4_mgd_STAR_srt.bam $4_lane1_STAR_srt.bam $4_lane2_STAR_srt.bam

# MAPQ filter step
samtools view -@ 8 -q 10 -b $4_mgd_STAR_srt.bam > $4_mgd_STAR_srt_q10.bam

#QUANTIFY GENE EXPRESSION WITH FEATURE COUNTS
featureCounts -T 16 -C -s 1 -t exon \\
-g gene_id -a /SPECIES_ENSEMBL_ANNOTATIONS/ \\
-o /OUTPUT_PATH/$4_feature_Counts.txt $4_mgd_STAR_srt_q10.bam 



Orthologous gene identification

In [ ]:
%%writefile 'GENE.EXPRESSION.ANALYSIS.R'

# Marco Trizzino

################### FIND GENES ANNOTATED AS ORTHOLOGOUS IN THE SIX STUDIED PRIMATES, USING BIOMART 
#In R

source("http://bioconductor.org/biocLite.R")
biocLite("biomaRt") #363kb
library(biomaRt)

ensembl <-  useMart("ensembl", dataset = "hsapiens_gene_ensembl")
listDatasets(ensembl)
listAttributes(ensembl)
myFilter <- "ensembl_gene_id"
myAttributes <- c("ensembl_gene_id",
                  "ogarnettii_homolog_ensembl_gene", 
                  "ptroglodytes_homolog_ensembl_gene",
                  "cjacchus_homolog_ensembl_gene",
                  "tbelangeri_homolog_ensembl_gene",
                  "mmurinus_homolog_ensembl_gene",
                  "mmulatta_homolog_ensembl_gene")
#The following step is slow:
GENEannot<- getBM(attributes =  myAttributes, 
                  filters =  myFilter,
                  values =  rownames(Nreads), 
                  mart = ensembl)
write.csv(file='ENSG_homologues.csv',GENEannot)
sum(GENEannot=="")
write.csv(file="ENSG_homologues_all.csv",
          GENEannot[GENEannot[,2]!="" & GENEannot[,3]!=""& GENEannot[,4]!=""& GENEannot[,5]!=""& GENEannot[,6]!=""& GENEannot[,7]!="",], 
          row.names=FALSE)


###################### 
#PREPARE MATRIX OF RNA-SEQ counts INCLUDING THE 10,243 ORTHOLOGOUS GENEs FOR EACH SPECIMEN

####FOR EACH INDIVIDUAL DO AS FOLLOWS (EXAMPLE WITH CHIMP CH_4X0430:
Nreads <-read.delim(file="CH_4X0430_RNA_feature_Counts.txt", header=T, row.names=NULL, sep="\t")
names(Nreads)[7]="CH_4X0430"
NreadsCOUNTS <- Nreads[,c(1,7)]
orthologs=read.csv("ENSG_homologues_all.csv",  header = TRUE)
index<-match(NreadsCOUNTS$Geneid,orthologs$ptroglodytes_homolog_ensembl_gene)
NreadsCOUNTS$HumanID=orthologs$ensembl_gene_id[index]
Counts_ortho= NreadsCOUNTS[!is.na(NreadsCOUNTS$HumanID),]
write.csv(file="CH_4X0430_counts_ortho.csv", Counts_ortho, row.names=FALSE)
#After processing all of the individual, 
#merge the individual csv files in a single one "GENE_COUNTS_ORTHOLOGOUS.csv": 
#each file (= read counts for a give specimen) becomes a column

###IMPORTANT: 
#Read counts need to be normalized by feature lengths before performing differential expression
#Example for CH_4X0519
lengths= read.csv("feature_legths_per_species_per_gene.csv", 
                  header=TRUE, row.names=NULL) 
#open matrix with the feature lengths per each of the species for each of the orthologous genes
lengths$CHIMP_factor <- lengths$HUMAN_lengths / lengths$CHIMP_lengths
counts=read.csv("GENE_COUNTS_ORTHOLOGOUS.csv", header = T)
counts$CH_4X0519_norm <- counts$CH_4X0519*counts$CHfactor



#### MATRIX WITH READ COUNTS FOR THE 10,683 ORTHOLOGOUS GENES, 
#NORMALIZED BY FEATURE LENGTHS FOR THE SIX SPECIES IS TABLE S3



###################### 
#PERFORM DIFFERENTIAL EXPRESSION WITH DEseq2 
#ON THE GENE COUNT MATRIX NORMALIZED BY FEATURE LENGTH (TABLE S3)

#Download DESeq2 from Bioconductor (only need to do the first time you use DESeq2):
source("http://bioconductor.org/biocLite.R")
biocLite("DESeq2")

#Load DESeq2- Do this step every time you open R and need to run DESeq2
library("DESeq2")

#Import your expression matrix- mine was a comma seperated values file
Nreads <-read.csv("GENE_COUNTS_ORTHOLOGOUS_NORM_FEAT_LENGTH.csv", header=TRUE, row.names=1)
#CRITICAL: You need to use read countsfor DESeq2, not normalized data like RPKMs (see manual).
#If read counts are not integers, they need to be round up to integers:
Nreads_norm=round(Nreads,0)
#Omit any genes with zero counts across samples:
Nreads <- Nreads[rowSums(Nreads)!=0,]
#genes that have count=0 in only some of the genes need to be an integer >0
Nreads=Nreads+1

#Now import a data frame that provides annotatios for the samples 
#The rows of this correspond to your samples, 
#column 1 has the name of the sample and column two the condition (=species or species group)
#for humans vs other primates:
colData1 <-read.csv("colData_HSvsOTHER.csv", header=TRUE, row.names=1) #HS VS ALL SPECIES
#for apes vs other primates:
colData2 <-read.csv("colData_APESvsALL.csv", header=TRUE, row.names=1) #APES VS ALL SPECIES
#for all species vs all species pair-wise comparisons:
colData3 <-read.csv("colData_ALL_SPECIES.csv", header=TRUE, row.names=1) #ALL SPECIES VS ALL SPECIES

###Now I want to look at 'HS' vs 'OTHERS' no fold change, FDR=0.05
#Perform DE analysis. You are telling it to compare by condition 
dds <- DESeqDataSetFromMatrix(countData = Nreads, colData=colData1)
dds <- DESeq(dds)

res_HS_OTHER<-results(dds, contrast=c("condition","HS","OTHER"))
#Remove those genes for which a multiple correction p-value is "NA"
res_HS_OTHER_p=res_HS_OTHER[!is.na(res_HS_OTHER$padj), ] 
#Only keep genes with corrected p-values below 0.05:
res_HS_OTHER_p=res_HS_OTHER_p[res_HS_OTHER_p$padj<0.05,] 
#Save logfile
write.csv(as.data.frame(res_HS_OTHER_p), file = "HSVsOTHER.csv")
#Get the names of upregulated genes (HS>OTHER):
HighHS_OTHER=rownames(res_HS_OTHER_p[res_HS_OTHER_p$log2FoldChange>0,])
write.csv(as.data.frame(HighHS_OTHER), file = "HighHS_OTHER.csv")
#Get the names of downregulated genes (HS<OTHER):
LowHS_OTHER=rownames(res_HS_OTHER_p[res_HS_OTHER_p$log2FoldChange<0,])
write.csv(as.data.frame(LowHS_OTHER), file = "LowHS_OTHER.csv")


###
#Now I want to look at 'APES' vs 'OTHERS' no fold change, FDR=0.05
dds <- DESeqDataSetFromMatrix(countData = Nreads, colData=colData2, design= ~condition)
dds <- DESeq(dds)

res_APES_OTHER<-results(dds, contrast=c("condition","APES","OTHER"))
#Remove those genes for which a multiple correction p-value is "NA"
res_APES_OTHER_p=res_APES_OTHER[!is.na(res_APES_OTHER$padj), ] 
#write.csv(as.data.frame(res_APES_OTHER_p), file = "APESVsOTHER_ALL_P.csv")
#Only keep genes with corrected p-values below 0.05:
res_APES_OTHER_p=res_APES_OTHER_p[res_APES_OTHER_p$padj<0.05,] 
#Get the names of upregulated genes (APES>OTHER):
HighAPES_OTHER=rownames(res_APES_OTHER_p[res_APES_OTHER_p$log2FoldChange>0,])
write.csv(as.data.frame(HighAPES_OTHER), file = "HighAPES_OTHER.csv")
#Get the names of downregulated genes (APES<OTHER):
LowAPES_OTHER=rownames(res_APES_OTHER_p[res_APES_OTHER_p$log2FoldChange<0,])
write.csv(as.data.frame(LowAPES_OTHER), file = "LowAPES_OTHER.csv")


###Now all of the possible human vs any other species pair-wise combinations no fold change, FDR=0.05

dds <- DESeqDataSetFromMatrix(countData = Nreads, colData=colData3, design= ~condition)
dds <- DESeq(dds)

# 'HS' vs 'BB'

res_HS_BB<-results(dds, contrast=c("condition","HS","BB"))
#Remove those genes for which a multiple correction p-value is "NA"
res_HS_BB_p=res_HS_BB[!is.na(res_HS_BB$padj), ] 
#Only keep genes with corrected p-values below 0.05:
res_HS_BB_p=res_HS_BB_p[res_HS_BB_p$padj<0.05,] 
#Get the names of upregulated genes (HS>BB):
HighHS_BB=rownames(res_HS_BB_p[res_HS_BB_p$log2FoldChange>0,])
write.csv(as.data.frame(HighHS_BB), file = "HighHS_BB.csv")
#Get the names of downregulated genes (HS<BB):
LowHS_BB=rownames(res_HS_BB_p[res_HS_BB_p$log2FoldChange<0,])
write.csv(as.data.frame(LowHS_BB), file = "LowHS_BB.csv")

#'HS' vs 'CH' 

res_HS_CH<-results(dds, contrast=c("condition","HS","CH"))
#Remove those genes for which a multiple correction p-value is "NA"
res_HS_CH_p=res_HS_CH[!is.na(res_HS_CH$padj), ] 
#Only keep genes with corrected p-values below 0.05:
res_HS_CH_p=res_HS_CH_p[res_HS_CH_p$padj<0.05,] 
#Get the names of upregulated genes (HS>CH):
HighHS_CH=rownames(res_HS_CH_p[res_HS_CH_p$log2FoldChange>0,])
write.csv(as.data.frame(HighHS_CH), file = "HighHS_CH.csv")
#Get the names of downregulated genes (HS<CH):
LowHS_CH=rownames(res_HS_CH_p[res_HS_CH_p$log2FoldChange<0,])
write.csv(as.data.frame(LowHS_CH), file = "LowHS_CH.csv")

#'HS' vs 'RH' 

res_HS_RH<-results(dds, contrast=c("condition","HS","RH"))
#Remove those genes for which a multiple correction p-value is "NA"
res_HS_RH_p=res_HS_RH[!is.na(res_HS_RH$padj), ] 
#Only keep genes with corrected p-values below 0.05:
res_HS_RH_p=res_HS_RH_p[res_HS_RH_p$padj<0.05,] 
#Get the names of upregulated genes (HS>RH):
HighHS_RH=rownames(res_HS_RH_p[res_HS_RH_p$log2FoldChange>0,])
write.csv(as.data.frame(HighHS_RH), file = "HighHS_RH.csv")
#Get the names of downregulated genes (HS<RH):
LowHS_RH=rownames(res_HS_RH_p[res_HS_RH_p$log2FoldChange<0,])
write.csv(as.data.frame(LowHS_RH), file = "LowHS_RH.csv")

#'HS' vs 'MS'

res_HS_MS<-results(dds, contrast=c("condition","HS","MS"))
#Remove those genes for whiMS a multiple correction p-value is "NA"
res_HS_MS_p=res_HS_MS[!is.na(res_HS_MS$padj), ] 
#Only keep genes with corrected p-values below 0.05:
res_HS_MS_p=res_HS_MS_p[res_HS_MS_p$padj<0.05,] 
#Get the names of upregulated genes (HS>MS):
HighHS_MS=rownames(res_HS_MS_p[res_HS_MS_p$log2FoldChange>0,])
write.csv(as.data.frame(HighHS_MS), file = "HighHS_MS.csv")
#Get the names of downregulated genes (HS<MS):
LowHS_MS=rownames(res_HS_MS_p[res_HS_MS_p$log2FoldChange<0,])
write.csv(as.data.frame(LowHS_MS), file = "LowHS_MS.csv")

#'HS' vs 'ML' 

res_HS_ML<-results(dds, contrast=c("condition","HS","ML"))
#Remove those genes for which a multiple correction p-value is "NA"
res_HS_ML_p=res_HS_ML[!is.na(res_HS_ML$padj), ] 
#Only keep genes with corrected p-values below 0.05:
res_HS_ML_p=res_HS_ML_p[res_HS_ML_p$padj<0.05,] 
#Get the names of upregulated genes (HS>ML):
HighHS_ML=rownames(res_HS_ML_p[res_HS_ML_p$log2FoldChange>0,])
write.csv(as.data.frame(HighHS_ML), file = "HighHS_ML.csv")
#Get the names of downregulated genes (HS<ML):
LowHS_ML=rownames(res_HS_ML_p[res_HS_ML_p$log2FoldChange<0,])
write.csv(as.data.frame(LowHS_ML), file = "LowHS_ML.csv")

#######################################################
#DIFFERENTIAL EXPRESSION ANALYSIS RESULTS ARE IN TABLES4

